In [2]:
import sys                  #menejo del sistema
import os                   #manejo del sistema operativo
import numpy as np          #libreria numerica de python
import tensorflow as tf     #libreria para redes profundas
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop

In [3]:
#ruta de el data set
mio=os.getcwd()
data_entrenamiento=os.path.join(mio+'/Data/Entrenamiento')
data_validacion=mio+'/Data/Validacion'

#definimos los parametros
epocas=10
image_size=(640,480)
batch_size=30
pasos=1000
pasos_validacion=200
lr=0.001

In [4]:
#Preporcesado del dataset
entrenamiento_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

validacion_datagen=ImageDataGenerator(rescale=1./255)

#importamos las imagenes y procesado
entrenamiento_generado=entrenamiento_datagen.flow_from_directory(
    data_entrenamiento,
    target_size=image_size,
    batch_size=50,
    class_mode='binary')

validacion_generador=validacion_datagen.flow_from_directory(
    data_validacion,
    target_size=image_size,
    batch_size=2,
    class_mode='binary')

Found 600 images belonging to 2 classes.
Found 24 images belonging to 2 classes.


In [5]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 640x480 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(640, 480, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('leucemia') and 1 for the other ('healt')
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 638, 478, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 319, 239, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 317, 237, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 158, 118, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 156, 116, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 78, 58, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 76, 56, 64)        3

In [9]:
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=lr),
              metrics=['accuracy'])

In [10]:
historia=model.fit(entrenamiento_generado,
    steps_per_epoch=12,
    epochs=10,
    validation_data=validacion_generador,
    validation_steps=12)

Epoch 1/10
12/12 [==============================] - 188s 14s/step - loss: 1.0004 - accuracy: 0.7715 - val_loss: 0.0457 - val_accuracy: 0.9583
Epoch 2/10
12/12 [==============================] - 196s 16s/step - loss: 0.0258 - accuracy: 0.9927 - val_loss: 3.3494 - val_accuracy: 0.5417
Epoch 3/10
12/12 [==============================] - 200s 16s/step - loss: 0.3385 - accuracy: 0.9353 - val_loss: 8.8060e-04 - val_accuracy: 1.0000
Epoch 4/10
12/12 [==============================] - 180s 15s/step - loss: 9.8640e-04 - accuracy: 1.0000 - val_loss: 1.8220e-04 - val_accuracy: 1.0000
Epoch 5/10
12/12 [==============================] - 179s 15s/step - loss: 1.9148e-04 - accuracy: 1.0000 - val_loss: 4.9769e-05 - val_accuracy: 1.0000
Epoch 6/10
12/12 [==============================] - 172s 14s/step - loss: 8.5670e-05 - accuracy: 1.0000 - val_loss: 1.4588e-05 - val_accuracy: 1.0000
Epoch 7/10
12/12 [==============================] - 167s 14s/step - loss: 8.6239e-05 - accuracy: 1.0000 - val_loss: 7.58

In [8]:
save_model=mio+'/Modelo'
if not os.path.exists(save_model):
    os.mkdir(save_model)
model.save(save_model+'/leucemiamodel.h5')
model.save_weights(save_model+'/leucemiapesos.h5')